# Experiment Tracking

We have done many experiments so far. It's getting really hard to track them. Moreover, they are in their individual notebooks. How do we summarize them altogether to get a sense of our progress? Let's go through a couple of solutions to this effect.

We'll follow the [PyTorch Experiment Tracking](https://www.learnpytorch.io/07_pytorch_experiment_tracking/) tutorial by [@mrdbourke](https://github.com/mrdbourke/pytorch-deep-learning)

# Import Modules

In [8]:
import random
import torch
import torchvision
import torchinfo

from toolbox import data_download, data_setup, engine, utils
from torchinfo import summary as model_summary
from timeit import default_timer as timer 
from pathlib import Path
from PIL import Image

print(f'torch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')
print(f'torchinfo version: {torchinfo.__version__}')

[INFO] /Users/broxoli/.datasets/pizza_steak_sushi directory exists. Skipping download.
Data Path: /Users/broxoli/.datasets/pizza_steak_sushi
torch version: 2.1.0.dev20230608
torchvision version: 0.16.0.dev20230608
torchinfo version: 1.8.0


# Device Selection

In [2]:
DEVICE = utils.get_device()

# Data Procurement

We will download the [Food101](https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip) dataset.

In [9]:
image_path = data_download.download_data(
    source='https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip',
    destination='pizza_steak_sushi',
)

print(f'Data Path: {image_path}')

[INFO] /Users/broxoli/.datasets/pizza_steak_sushi directory exists. Skipping download.
Data Path: /Users/broxoli/.datasets/pizza_steak_sushi


# Create Datasets and DataLoaders

Steps:
1. Setup Hyperparameters
2. Choose a pretrained model for weights.
3. Load transforms from the pretrained model.
4. Initialize Dataloaders.

## Hyperparameters

In [11]:
BATCH_SIZE = 32

## Load Transforms from the Pretrained Model

In [15]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
# weights = torchvision.models.ResNet50_Weights.DEFAULT

print(f'Pretrained Weights: {weights}')

transform = weights.transforms()

print(f'Transform: {transform}')

Pretrained Weights: EfficientNet_B0_Weights.IMAGENET1K_V1
Transform: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


## Initialize DataLoaders

In [16]:
# 1. Setup train and test paths
train_path = image_path / "train"
test_path = image_path / "test"

# 2. Create train and test dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_path=train_path,
    test_path=test_path,
    transform=transform,
    batch_size=BATCH_SIZE
)

Training Set
------------
Dataset ImageFolder
    Number of datapoints: 225
    Root location: /Users/broxoli/.datasets/pizza_steak_sushi/train
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
               resize_size=[256]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BICUBIC
           )
Classes: ['pizza', 'steak', 'sushi']
Class Dictionary: {'pizza': 0, 'steak': 1, 'sushi': 2}

Test Set
------------
Dataset ImageFolder
    Number of datapoints: 75
    Root location: /Users/broxoli/.datasets/pizza_steak_sushi/test
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
               resize_size=[256]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BICUBIC
           )
Classes: ['pizza', 'steak', 'sushi']
Class Dictionary: {'pizza': 0, 'steak': 1, 'sushi'

# Model Design

## Get a Pretrained Model

In [18]:
model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT).to(DEVICE)
# model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT).to(DEVICE)

model_summary(
    model=model, 
    input_size=(32, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 1000]           --                   True
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   9,408                True
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   128                  True
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   True
│    └─Bottleneck (0)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   True
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     4,096                True
│    │    └─BatchN

## Freeze Base Layers

In [19]:
# Freeze all weights except the last two layers
for m in list(model.children())[:-2]:
    for p in m.parameters():
        p.requires_grad = False

model_summary(
    model=model, 
    input_size=(32, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 1000]           --                   Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    └─Bottleneck (0)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     (4,096)              False
│    │    

## Attach a Classifier

In [22]:
# from importlib import reload

# reload(utils)

# 1. Set random seed
utils.set_seeds(3)

# 2. Get the size of the classification head
output_shape = len(class_names)

# Create a classification head
model.fc = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(
        in_features=2048, 
        out_features=output_shape,
        bias=True
    )
).to(DEVICE)

model_summary(
    model=model, 
    input_size=(32, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    └─Bottleneck (0)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     (4,096)              False
│    │    

# Training

## Hyperparameters

In [25]:
LEARNING_RATE = 0.001
EPOCHS = 5

## Loss Function and Optimizer

In [26]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lr=LEARNING_RATE, params=model.parameters())

## Loop

In [47]:
print('Training Loop')
print('-------------')
print('-------------')

# 1. Record run start_time
start_time = timer()

# 2. Create a summary writer to log training metrics
writer = create_writer(
    experiment_name='getting_started',
    model_name='resnet_pretrained'
)

# 3. Traiing
results = train(
    model=model.to(DEVICE),
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    accuracy_fn=utils.accuracy_fn,
    epochs=EPOCHS,
    device=DEVICE,
    writer=writer
)

# 4. Record end_time
end_time = timer()

print(f"Total training time: {end_time - start_time:.3f} seconds")

Training Loop
-------------
-------------
[INFO] Created a SummaryWriter writing to runs/2023-07-21/getting_started/resnet_pretrained


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5198 | train_acc: 0.8125 | test_loss: 0.6094 | test_acc: 0.8769
Epoch: 2 | train_loss: 0.4740 | train_acc: 0.8359 | test_loss: 0.6351 | test_acc: 0.9081


KeyboardInterrupt: 